In [ ]:
import osiris_utils as ou
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import matplotlib as mpl
import pandas as pd
from tqdm import tqdm
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.colors as mcolors
plt.rcParams['font.size'] = 14
from scipy.stats import linregress
from matplotlib.ticker import FuncFormatter, MaxNLocator, FixedLocator,  FormatStrFormatter

from mpl_toolkits.mplot3d import Axes3D

import contextlib
import io


In [ ]:
# Normalize axis to w_ce

def _set_scaled_formatter(axis, scale_factor, fmt=".2f"):
    axis.set_major_formatter(
        FuncFormatter(lambda v, pos: f"{v*scale_factor:{fmt}}")
    )

def scale_x_ax(scale_factor, fig, ax, label=r"$t[2\pi / \Omega_e]$", lock_ticks=False, fmt=".2f"):
    ax.set_xlabel(label)
    if lock_ticks:
        # freeze current tick positions
        ticks = ax.get_xticks()
        ax.xaxis.set_major_locator(FixedLocator(ticks))
    _set_scaled_formatter(ax.xaxis, scale_factor, fmt)
    return fig, ax

def scale_y_ax(scale_factor, fig, ax, label=r"$x[c / \Omega_e]$", lock_ticks=False, fmt=".2f"):
    ax.set_ylabel(label)
    if lock_ticks:
        ticks = ax.get_yticks()
        ax.yaxis.set_major_locator(FixedLocator(ticks))
    _set_scaled_formatter(ax.yaxis, scale_factor, fmt)
    return fig, ax

def scale_z_ax(scale_factor, fig, ax, label=r"$x[c / \Omega_e]$", lock_ticks=False, fmt=".2f"):
    ax.set_zlabel(label)
    if lock_ticks:
        ticks = ax.get_zticks()
        ax.zaxis.set_major_locator(FixedLocator(ticks))
    _set_scaled_formatter(ax.zaxis, scale_factor, fmt)
    return fig, ax

def scale_3d_axes(scale_factor, fig, ax, fmt=".2f"):
    ax.set_xlabel(r"$x1[c / \Omega_e]$")
    ax.set_ylabel(r"$x2[c / \Omega_e]$")
    ax.set_zlabel(r"$x3[c / \Omega_e]$")
    _set_scaled_formatter(ax.xaxis, scale_factor, fmt)
    _set_scaled_formatter(ax.yaxis, scale_factor, fmt)
    _set_scaled_formatter(ax.zaxis, scale_factor, fmt)
    return fig, ax


In [ ]:
def call_silently(func, *args, **kwargs):
    with contextlib.redirect_stdout(io.StringIO()):
        return func(*args, **kwargs)


def get_trajectory(sim, particle, tmin = 0, tmax = 100000000000, unload=False):
    track = sim["test_electrons"]["tracks"]
    track.load_all()

    mask = (track.data["t"][particle, :] >= tmin) & (track.data["t"][particle, :] <= tmax)
    traj = np.stack([
        track.data["x1"][particle, :][mask],
        track.data["x2"][particle, :][mask],
        track.data["x3"][particle, :][mask],
        track.data["t"][particle, :][mask]
        ], axis=0)

    if unload:
        track.unload()

    return traj

def plot_trajectory(sim, label, particle, avrg_B, tmin = 0, tmax = 100000000000, unload=False, fig=None, ax=None):
    traj = call_silently(get_trajectory, sim, particle, tmin, tmax)

    x, y, z, t = traj[0, :], traj[1, :], traj[2, :], traj[3, :]
    
    if fig is None:
        fig = plt.figure(figsize=(10, 7))

    label1 = None
    label2 = None
    if ax is None:
        ax = fig.add_subplot(111, projection='3d')
        label1 = ("$t = {:.3f}$".format(t[0] * avrg_B / 2.0 / np.pi) + r"$ [2\pi / \Omega_e]$")
        label2 = ("$t = {:.3f}$".format(t[-1] * avrg_B / 2.0 / np.pi)  + r"$[ 2\pi / \Omega_e]$")


    if label == "Baseline":
        color = "black"
        markersize = 0
        linewidth = 0.6
    else:
        color = None
        markersize = 1.5
        linewidth = 0.8
    
    # Highlight start and end points
    ax.scatter(x[0], y[0], z[0], color='lightgreen', s=20, label=label1)
    ax.scatter(x[-1], y[-1], z[-1], color='r', s=20, label=label2)

    # Plot the trajectory
    ax.plot(x, y, z, marker='o', linestyle='-', markersize=markersize, linewidth=linewidth, label=label, color=color)
    
    # Labels and title
    fig, ax = scale_3d_axes(avrg_B, fig=fig, ax=ax)
    ax.legend()

    return fig, ax

In [ ]:
# raw = ou.OsirisRawFile("/home/exxxx5/Tese/Decks/GCA_PusherTest/2d/MS/RAW/electrons/RAW-electrons-000003.h5")
# raw.raw_to_file_tags("/home/exxxx5/Tese/Decks/GCA_PusherTest/2d/file_tags.tags", type="random", n_tags=5)

## Gyromotion 3d

In [ ]:
gca_gyro_1000 = ou.Simulation("/home/exxxx5/Tese/Decks/MethodicTests/Gyro/Gca/dtw1000/Gca.in")
gca_gyro_100 = ou.Simulation("/home/exxxx5/Tese/Decks/MethodicTests/Gyro/Gca/dtw100/Gca.in")

gca_corr_gyro_1000 = ou.Simulation("/home/exxxx5/Tese/Decks/MethodicTests/Gyro/GcaCorr/dtw1000/GcaCorr.in")
gca_corr_gyro_100 = ou.Simulation("/home/exxxx5/Tese/Decks/MethodicTests/Gyro/GcaCorr/dtw100/GcaCorr.in")

boris_gyro_1000 = ou.Simulation("/home/exxxx5/Tese/Decks/MethodicTests/Gyro/Boris/dtw1000/Boris.in")
boris_gyro_100 = ou.Simulation("/home/exxxx5/Tese/Decks/MethodicTests/Gyro/Boris/dtw100/Boris.in")

In [ ]:
particle = 0
avrg_B = 1000.0

fig, ax = plt.subplots()
ax.plot(boris_gyro_100["test_electrons"]["tracks"]["x2"][particle,:], boris_gyro_100["test_electrons"]["tracks"]["x3"][particle,:3500], label="Boris", marker='o', linestyle='-', markersize=1.5, linewidth=0.8)
ax.plot(boris_gyro_1000["test_electrons"]["tracks"]["x2"][particle,:], boris_gyro_1000["test_electrons"]["tracks"]["x3"][particle,:], label="Boris", marker='o', linestyle='-', markersize=1.5, linewidth=0.8)

ax.plot(gca_gyro_100["test_electrons"]["tracks"]["x2"][particle,:], gca_gyro_100["test_electrons"]["tracks"]["x3"][particle,:3500], label="GCA", marker='o', linestyle='-', markersize=1.5, linewidth=0.8)
ax.plot(gca_gyro_1000["test_electrons"]["tracks"]["x2"][particle,:], gca_gyro_1000["test_electrons"]["tracks"]["x3"][particle,:], label="GCA", marker='o', linestyle='-', markersize=1.5, linewidth=0.8)

ax.plot(gca_corr_gyro_100["test_electrons"]["tracks"]["x2"][particle,:], gca_corr_gyro_100["test_electrons"]["tracks"]["x3"][particle,:], label="GCA Corr 100 ", marker='o', linestyle='-', markersize=1.5, linewidth=0.8)
ax.plot(gca_corr_gyro_1000["test_electrons"]["tracks"]["x2"][particle,:], gca_corr_gyro_1000["test_electrons"]["tracks"]["x3"][particle,:], label="GCA Corr 1000", marker='o', linestyle='-', markersize=1.5, linewidth=0.8)

ax.xaxis.set_major_locator(plt.MaxNLocator(4))
ax.yaxis.set_major_locator(plt.MaxNLocator(4))

fig, ax = scale_x_ax(avrg_B, label = r"$x2[c / \Omega_e]$", fig=fig, ax=ax, fmt=".0f")
fig, ax = scale_y_ax(avrg_B, label = r"$x3[c / \Omega_e]$", fig=fig, ax=ax, fmt=".0f")


ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))


plt.show()




# fig, ax = None, None
# fig, ax = plot_trajectory(boris, r"Boris", particle=particle, avrg_B=avrg_B, tmax=50, fig=fig, ax=ax)
# fig, ax = plot_trajectory(gca, r"GCA", particle=particle, avrg_B=avrg_B, tmax=50, fig=fig, ax=ax)
# fig, ax = plot_trajectory(gca_corr, r"GCA corr", particle=particle, avrg_B=avrg_B, tmax=50, fig=fig, ax=ax)


# fig, ax = plt.subplots()
# ax.plot(boris["test_electrons"]["tracks"]["t"][particle,:], boris["test_electrons"]["tracks"]["x1"][particle,:], label="boris")
# ax.plot(gca["test_electrons"]["tracks"]["t"][particle,:], gca["test_electrons"]["tracks"]["x1"][particle,:], label="gca")
# ax.plot(gca_corr["test_electrons"]["tracks"]["t"][particle,:], gca_corr["test_electrons"]["tracks"]["x1"][particle,:], label="gca_corr")


# ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))


# fig, ax = scale_x_ax(avrg_B / 2.0 / np.pi, fig=fig, ax=ax)
# fig, ax = scale_y_ax(avrg_B, label=r"$x1[c / \Omega_e]$",fig=fig, ax=ax)

plt.show()